In [1]:
import pandas as pd

import nltk
from nltk.tokenize import TreebankWordTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score

In [2]:
# 전처리 끝낸 데이터 불러오기
raw = pd.read_csv('/Users/mac/AIFFEL/hackathon/new_musinsa(upcycling).csv', engine='python')
print(raw.shape)
raw.head()

(715, 12)


,Unnamed: 0,user,color,contents,category,ratings,gender,price,style,up_check,brand,url
0,0,멤버_464f9cc84f7d,기타,면,상의,5,남자,39000,0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,1,멤버_464f9cc84f7d,카키,면,하의,5,남자,42000,0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,2,멤버_464f9cc84f7d,기타,면,모자,5,남자,59000,0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,3,멤버_464f9cc84f7d,블랙,면,상의,5,남자,35000,0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,4,멤버_464f9cc84f7d,아이보리,면 아라미드,가방,5,남자,48000,1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


In [3]:
# 불필요한 컬럼 제거
raw.drop(['Unnamed: 0', 'user', 'up_check', 'url'], axis=1, inplace=True)
raw.head()

,color,contents,category,ratings,gender,price,style,brand
0,기타,면,상의,5,남자,39000,0,119레오
1,카키,면,하의,5,남자,42000,0,119레오
2,기타,면,모자,5,남자,59000,0,119레오
3,블랙,면,상의,5,남자,35000,0,119레오
4,아이보리,면 아라미드,가방,5,남자,48000,1,119레오


In [4]:
# rating 2점 이상만 사용
orginal_data_size = len(raw)
raw = raw[raw['ratings']>=2]
filtered_data_size = len(raw)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 715, filtered_data_size: 714
Ratio of Remaining Data is 99.86%


In [6]:
# user 와 brand 를 제외한 컬럼을 features 로 합치기
cols = ['color', 'contents', 'category', 'ratings', 'gender', 'price', 'style']
raw['features'] = raw[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
raw.drop(cols, axis=1, inplace=True)
raw

,brand,features
0,119레오,기타 면 상의 5 남자 39000 0
1,119레오,카키 면 하의 5 남자 42000 0
2,119레오,기타 면 모자 5 남자 59000 0
3,119레오,블랙 면 상의 5 남자 35000 0
4,119레오,아이보리 면 아라미드 가방 5 남자 48000 1
...,...,...
710,플리츠마마,기타 폴리에스터 폴리우레탄 가방 5 여자 59000 0
711,플리츠마마,아이보리 울 아크릴 모자 5 여자 35000 0
712,플리츠마마,베이지 아크릴 모자 5 여자 29000 0
713,플리츠마마,블랙 면 폴리에스터 상의 5 여자 138000 0


In [7]:
# Test set / Train set 나누기
X_train, X_test, y_train, y_test = train_test_split(raw.features, raw.brand, test_size=0.2, random_state=4)

print('#Train set size: ', len(X_train))
print('#Test set size: ', len(X_test))

#Train set size:  571
#Test set size:  143


In [8]:
X_train[:10]

321              기타 면 상의 5 남자 27500 0
80         그린 면 폴리에스터 상의 5 여자 59000 0
463              블랙 면 상의 5 남자 39000 1
615           블랙 코듀라 가방 5 여자 119000 0
551           화이트 면 상의 5 알수없음 39000 0
533            아이보리 면 하의 5 남자 69000 0
421    기타 폴리에스터 폴리우레탄 가방 5 남자 88000 0
165          멀티 광고현수막 가방 5 여자 79000 1
190             블랙 면 하의 5 남자 109000 1
659          블랙 폴리에스터 가방 5 여자 59800 0
Name: features, dtype: object

In [9]:
X_test[:10]

384                   블랙 나일론 모자 5 남자 49900 0
262                  그린 나일론 상의 4 남자 395000 0
112    화이트 폴리에스터 레이온 폴리우레탄 상의 5 알수없음 12900 0
430                그레이 폴리에스터 모자 5 남자 27000 0
195          베이지 면 나일론 폴리우레탄 하의 5 남자 89000 0
33                     네이비 면 하의 5 여자 67000 0
604                 기타 폴리에스터 셋업 5 남자 55000 0
229                  화이트 면 상의 5 알수없음 49000 0
155               화이트 None 상의 5 알수없음 58000 0
83            블랙 폴리에스터 폴리우레탄 셋업 5 여자 92000 0
Name: features, dtype: object

In [10]:
# 토크나이저 사용하여 토큰화
tb_tokenizer = TreebankWordTokenizer()

# TF-IDF 벡터화 
tfidf = TfidfVectorizer(tokenizer=tb_tokenizer.tokenize, max_features=2000)

X_train_tfidf = tfidf.fit_transform(X_train)  # train set 을 변환
X_test_tfidf = tfidf.transform(X_test)  # test set 을 변환

print(X_train_tfidf[:10])
print()
print(X_test_tfidf[:10])

  (0, 0)	0.1459803129111718
  (0, 69)	0.8135805985198138
  (0, 186)	0.23253590034103686
  (0, 119)	0.13625637100236562
  (0, 201)	0.21813894863670513
  (0, 195)	0.20358598821633128
  (0, 184)	0.39383468468127536
  (1, 133)	0.6230064942617041
  (1, 207)	0.2618574915024688
  (1, 219)	0.2871286146360805
  (1, 183)	0.5446487506449104
  (1, 0)	0.1647601871990626
  (1, 119)	0.15378529300094843
  (1, 201)	0.24620178773462353
  (1, 195)	0.2297766381007797
  (2, 1)	0.4533810063674035
  (2, 97)	0.6091743469672026
  (2, 199)	0.35161463814332866
  (2, 186)	0.31661532537771425
  (2, 119)	0.18552341886319584
  (2, 201)	0.29701277995729086
  (2, 195)	0.2771978167969873
  (3, 13)	0.4773422579060124
  (3, 180)	0.3088076049646628
  (3, 212)	0.7380880782242846
  :	:
  (6, 0)	0.13199726369946999
  (6, 186)	0.21026193152214098
  (6, 119)	0.12320475121104946
  (6, 184)	0.3561103527672568
  (7, 157)	0.46504773223959994
  (7, 181)	0.5580882326491367
  (7, 194)	0.4568490270554881
  (7, 180)	0.31950755364449773

In [11]:
# sklearn 에서 제공하는 Decision Tree 를 이용
tree = DecisionTreeClassifier(random_state=7)  # 분류기 선언

# train data를 이용해 분류기를 학습
tree.fit(X_train_tfidf, y_train)

# score 확인
print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

#Decision Tree train set score: 0.991
#Decision Tree test set score: 0.392


In [12]:
# 오차행렬, 정확도, 정밀도, 재현율을 한번에 출력하기 위한 함수 정의

pred = tree.predict(X_test_tfidf)

def get_tree_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    print('Confusion Matrix')
    print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{}'.format(accuracy, precision, recall))

In [13]:
get_tree_eval(y_test, pred)

Confusion Matrix
[[ 5  0  2  0  1  4]
 [ 5 13  9  1  2  8]
 [ 2  9 18  4  0  3]
 [ 0  2  3  0  0  0]
 [ 0  0  2  1  1  0]
 [ 5 11  6  3  4 19]]
정확도:0.3916083916083916, 정밀도:0.3916083916083916, 재현율:0.3916083916083916


In [14]:
# f1-score
f1 = f1_score(y_test, pred, average='micro')
print('f1 score :', f1)

f1 score : 0.3916083916083916


In [15]:
print('실제브랜드, 예측한브랜드, features')
for content in zip(y_test[:10], tree.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)

실제브랜드, 예측한브랜드, features
('오버랩업사이클', '오버랩업사이클', '블랙 나일론 모자 5 남자 49900 0')
('오버랩업사이클', '누깍', '그린 나일론 상의 4 남자 395000 0')
('누깍', '오버랩업사이클', '화이트 폴리에스터 레이온 폴리우레탄 상의 5 알수없음 12900 0')
('오버랩업사이클', '오버랩업사이클', '그레이 폴리에스터 모자 5 남자 27000 0')
('누깍', '누깍', '베이지 면 나일론 폴리우레탄 하의 5 남자 89000 0')
('119레오', '플리츠마마', '네이비 면 하의 5 여자 67000 0')
('플리츠마마', '누깍', '기타 폴리에스터 셋업 5 남자 55000 0')
('누깍', '플리츠마마', '화이트 면 상의 5 알수없음 49000 0')
('누깍', '오버랩업사이클', '화이트 None 상의 5 알수없음 58000 0')
('누깍', '플리츠마마', '블랙 폴리에스터 폴리우레탄 셋업 5 여자 92000 0')
